0) Imports:

In [19]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter

1) Configuration:

2. Load PDF file:

In [20]:
pdf_file = open("data/servicios_ebook.pdf", "rb")
pdf_reader = PdfReader(pdf_file)

In [21]:
text = ""
for page in pdf_reader.pages:
    text += page.extract_text()

In [22]:
#Check a part of the pdf extracted:
text[1000:2000]

'ebes tener millones de dudas, porque como te comentamos, también estuvimos en tu lugar\nhace muchos años.\nPor todo lo que te comentamos, también abrimos esta vía de consultoría, donde nos puedes\npreguntar de todo (o casi todo) ya que las cuestiones legales y migratorias por ley , solo lo puede\ndar un Inmigration Adviser . Nosotros contamos con uno, y puedes ver sus servicios de visas\npara Nueva Zelanda . Si tienes consultas migratorias para Australia, nos puedes escribir a\ncontacto@destinoceania.com .\n¿En qué te podemos ayudar? Te contamos brevemente nuestra historia, tal vez te ayude a\ndecidirte con quién de nosotros tener la consultoría.Menú\n11/2/23, 6:40 PM Consultoría – Destino Oceanía\nhttps://destinoceania.com/servicios/consultoria/ 2/4\nPatricio Toranza\nCo-fundador de Destino Oceanía, especialista de Nueva Zelanda\nPato pasó por varios trabajos en la construcción mediante los cuales consiguió su\nresidencia permanente en Nueva Zelanda para luego montarse su propia empr

3. Create chunks:

In [23]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
)

In [24]:
chunks = text_splitter.split_text(text)

In [25]:

print(f"Number of chunks: {len(chunks)}")
print(f"Example chunk[7]: {chunks[7]}")

Number of chunks: 53
Example chunk[7]: Nueva Zelanda puede abrirte el camino para desarrollar tus habilidades, conocer gente de todo el mundo e insertarse
en el incipiente mercado de trabajo de este país. Te asistimos para conseguir tu V isa de Estudiante y que puedas
viajar a Nueva Zelanda.
Gestión de visas de Visitante para Nueva Zelanda
Las visas de visitantes son para aquellos ciudadanos de países no miembros del  NZ V isa W aiver program ,
y permiten viajar a Nueva Zelanda con propósitos tales como vacaciones, visitar familiares, amigos (incluso estudiarMenú
Índice de contenidos11/2/23, 6:41 PM Servicio de gestión de visas para Nueva Zelanda
https://destinoceania.com/servicios/gestion-visas-nueva-zelanda/ 2/3hasta un máximo de 3 meses). Las visas de visitante permiten normalmente una estadía máxima de 9 meses en
Nueva Zelanda y son el puntapié inicial para descubrir los increíbles lugares de este país.
Si estás interesado en los servicios de gestión de visas para Nueva Zelanda, po

4. Create Embeddings: